![Enedis](https://www.fournisseurs-electricite.com/sites/fournisseurs-electricite.com/files/2019-12/enedis-presentation.png)

### Import packages

In [ ]:
import pandas as pd
import requests
import json
import csv
import datetime
import time

: 

### Create an empty csv

In [56]:
import csv
def create_csv():
    header = ["courbe_moyenne_ndeg2_wh",
            "region",
            "indice_representativite_courbe_ndeg1",
            "indice_representativite_courbe_ndeg1_ndeg2",
            "horodate",
            "jour_max_du_mois_0_1",
            "semaine_max_du_mois_0_1",
            "profil",
            "code_region",
            "plage_de_puissance_souscrite",
            "indice_representativite_courbe_ndeg2",
            "nb_points_soutirage",
            "courbe_moyenne_ndeg1_ndeg2_wh",
            "total_energie_soutiree_wh",
            "courbe_moyenne_ndeg1_wh"]

    with open("output.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)

### Function to get data from API, this function will save results in the same csv created later

In [60]:
def request_get(url):
    # Make API request
    response = requests.get(url)

    # Check if response is successful
    if response.status_code == 200:
        # Parse JSON data from response
        data = response.json()

        # Open existing CSV file in write mode
        with open('output.csv', mode='a', newline='') as csv_file:
            # Create CSV writer object
            writer = csv.writer(csv_file)

            # Write data to CSV file
            for item in data['records']:
                    try:
                        courbe_moyenne_ndeg2_wh = item['fields']['courbe_moyenne_ndeg2_wh']
                    except:
                        courbe_moyenne_ndeg2_wh = None

                    try:
                        region = item['fields']['region']
                    except:
                        region = None

                    try:
                        indice_representativite_courbe_ndeg1 = item['fields']['indice_representativite_courbe_ndeg1']
                    except:
                        indice_representativite_courbe_ndeg1 = None

                    try:
                        indice_representativite_courbe_ndeg1_ndeg2 = item['fields']['indice_representativite_courbe_ndeg1_ndeg2']
                    except:
                        indice_representativite_courbe_ndeg1_ndeg2 = None

                    try:
                        horodate = item['fields']['horodate']
                    except:
                        horodate = None

                    try:
                        jour_max_du_mois_0_1 = item['fields']['jour_max_du_mois_0_1']
                    except:
                        jour_max_du_mois_0_1 = None

                    try:
                        semaine_max_du_mois_0_1 = item['fields']['semaine_max_du_mois_0_1']
                    except:
                        semaine_max_du_mois_0_1 = None

                    try:
                        profil = item['fields']['profil']
                    except:
                        profil = None

                    try:
                        code_region = item['fields']['code_region']
                    except:
                        code_region = None

                    try:
                        plage_de_puissance_souscrite = item['fields']['plage_de_puissance_souscrite']
                    except:
                        plage_de_puissance_souscrite = None

                    try:
                        indice_representativite_courbe_ndeg2 = item['fields']['indice_representativite_courbe_ndeg2']
                    except:
                        indice_representativite_courbe_ndeg2 = None

                    try:
                        nb_points_soutirage = item['fields']['nb_points_soutirage']
                    except:
                        nb_points_soutirage = None

                    try:
                        courbe_moyenne_ndeg1_ndeg2_wh = item['fields']['courbe_moyenne_ndeg1_ndeg2_wh']
                    except:
                        courbe_moyenne_ndeg1_ndeg2_wh = None

                    try:
                        total_energie_soutiree_wh = item['fields']['total_energie_soutiree_wh']
                    except:
                        total_energie_soutiree_wh   = None

                    try:
                        courbe_moyenne_ndeg1_wh = item['fields']['courbe_moyenne_ndeg1_wh']
                    except:
                        courbe_moyenne_ndeg1_wh = None                                     


                    writer.writerow([ courbe_moyenne_ndeg2_wh,    region,    indice_representativite_courbe_ndeg1,    indice_representativite_courbe_ndeg1_ndeg2,    horodate,    jour_max_du_mois_0_1,    semaine_max_du_mois_0_1,    profil,    code_region,    plage_de_puissance_souscrite,    indice_representativite_courbe_ndeg2,    nb_points_soutirage,    courbe_moyenne_ndeg1_ndeg2_wh,    total_energie_soutiree_wh,    courbe_moyenne_ndeg1_wh])
    else:
        print('API request failed with status code', response.status_code)


## Main function

- Call function to create csv
- Loop over date and hour
- Call API
- Save results to the same csv

In [ ]:

create_csv()

base_url = "https://data.enedis.fr/api/records/1.0/search/?dataset=conso-inf36-region&q={}&rows=-1&facet=horodate&facet=region&facet=profil&facet=plage_de_puissance_souscrite&facet=indice_representativite_courbe_ndeg1&facet=indice_representativite_courbe_ndeg2&facet=indice_representativite_courbe_ndeg1_ndeg2&facet=jour_max_du_mois_0_1&facet=semaine_max_du_mois_0_1"

start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)
delta = datetime.timedelta(hours=1)

current_date = start_date
while current_date <= end_date:
    current_hour = datetime.datetime.combine(current_date, datetime.time.min)
    while current_hour <= datetime.datetime.combine(current_date, datetime.time.max):
        query_string = "horodate%3A%5B{}Z+TO+{}Z%5D".format(current_hour.isoformat(), (current_hour+delta).isoformat())
        url = base_url.format(query_string)
        
        
        
        request_get(url)



        print(f"Done: {current_hour}")
        # process the response as needed
        time.sleep(0.5)
        current_hour += delta


    time.sleep(0.5)
    current_date += datetime.timedelta(days=1)

### Read data

In [ ]:
data = pd.read_csv("data/consommation_electrique/output_2022.csv")

### Split data to mutilple csv by day

In [ ]:
# Convert the horodate column to a datetime object
data['horodate'] = pd.to_datetime(data['horodate'])

# Group the data by the horodate column
groups = data.groupby(pd.Grouper(key='horodate', freq='D'))

# Loop through the groups and write each group to a separate CSV file
for name, group in groups:
    filename = 'data/consommation_electrique/' + name.strftime('%Y-%m-%d') + '.csv'
    group.to_csv(filename, index=False)
